## Import your stuff

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import simulators.faas.simulator as faas
import pydelfi.ndes as ndes
import pydelfi.delfi as delfi
import pydelfi.score as score
import pydelfi.priors as priors
import tensorflow as tf
import rpy2.robjects.numpy2ri
from rpy2.robjects import rinterface, r, IntVector, FloatVector, StrVector
import logging
logging.getLogger('faas').setLevel(logging.ERROR)
logging.getLogger('delfi').setLevel(logging.ERROR)
rpy2.robjects.numpy2ri.activate()
tf.logging.set_verbosity(tf.logging.ERROR)
%matplotlib inline
seed = 20

In [2]:
#get batch arguments
batch = 'Whole'
if batch not in ['A','B','C','D','E','F','G', 'Whole']:
    print("Pass a valid batch letter")


batch_ids = {'A':[0,12,'A'],'B':[13,23,'B'],'C':[24,34,'C'],'D':[35,50,'D'],'E':[51,64,'E'],'F':[65,79,'F'],'G':[80,93,'G'], 'Whole': [0,93,'Whole']}
batch_min, batch_max, let = batch_ids[batch][0], batch_ids[batch][1], batch_ids[batch][2]

## Set up the prior

Last working prior
upper = np.array([7.700001e+08,  8.400001e+07,  3.200001e+10,  2.500001e+07,  1.600001e+05,
  2.600001e+03,  2.200001e+04,  6.500001e+00,  8.886492e+00,  7.924280e+00,
  1.050516e+01,  7.397941e+00, -3.682371e+00, -4.509306e+00, -6.162727e+00,
 -6.585027e+00,  1.100001e-03, -3.900000e-01])
lower = np.array([7.700000e+08,  8.400000e+07,  3.200000e+10,  2.500000e+07,  1.600000e+05,
  2.600000e+03,  2.200000e+04,  6.500000e+00,  8.886491e+00,  7.924279e+00,
  1.050515e+01,  7.397940e+00, -3.682372e+00, -4.509307e+00, -6.162728e+00,
 -6.585028e+00,  1.100000e-03, -3.900001e-01])

In [3]:
upper = np.array([12,  12,
  12,  12, -2, -2, -2,
 -2])
lower = np.array([6,  6,
  6,  6, -8, -8, -8,
 -8])

prior_mean = np.array([1.1e-03, -3.9e-01])
prior_cov = np.diag(np.array([(0.0011*0.2)**2, (-0.39*0.2)**2]))

prior = priors.Gaussian_Unif(prior_mean, prior_cov, lower, upper, 2)

epsilon_mean = np.array([0]*94)
epsilon_cov = np.diag(np.array([0.1 ** 2] * 94))
epsilon_prior = priors.Gaussian(epsilon_mean, epsilon_cov)

## Set up the simulator
This must have the signature `simulator(parameters, seed, args, batch)` -> `np.array([batch, ndata])`

In [4]:
faasSimulator = faas.faas_Model()

## repeating last fratio for short rows
def simulator(theta, seed, simulator_args, batch):

    eps_prior = simulator_args[0]
    eps = eps_prior.draw()

    return faasSimulator.simulation(np.concatenate([theta, eps]), seed)[batch_min:batch_max,1:]

simulator_args = [epsilon_prior]


## Set up the compressor
Must have the signature `compressor(data, args)` -> `np.array([n_summaries])`<br>
In this case we are going to do Gaussian score compression $$\mathbf{t} = \boldsymbol\theta_* + \mathbf{F}^{-1}\nabla_\theta^T\boldsymbol\mu_*\mathbf{C}^{-1}(\mathbf{d}-\boldsymbol\mu_*)$$ using the class `score.Gaussian`. For this we'll need some fiducial parameters, the mean its derivative at the fiducial parameters, the inverse covariance, and the inverse Fisher matrix

In [5]:
def compressor(d, compressor_args):
    comp_d = []

    for i in d:
        #mean of final ten entries
        final = np.mean(i[-10:])
        #median of 3 highest points
        peak = np.median(np.sort(i)[-3:])
        #time to peak
        time_peak = np.argmax(i == peak)
        sd = np.std(i[-10:])
        #time to final
        time_final = np.argmax(np.logical_and((i[time_peak:] >= final - sd),(i[time_peak:] <= final + sd)) == True) + time_peak
        comp_d.append([final, peak, time_peak, time_final])
    return np.asarray(comp_d).flatten()
compressor_args=None




## Load in the compressed data

In [6]:
compressed_data = compressor(faasSimulator.data()[batch_min:batch_max], compressor_args)

ndata = len(compressed_data)


## Load in data

In [7]:
data = faasSimulator.data()[:12,:]


## Define ensemble of NDEs

In [8]:
NDEs = [ndes.ConditionalMaskedAutoregressiveFlow(n_parameters=10, n_data=ndata, n_hiddens=[50,50], n_mades=5, act_fun=tf.tanh, index=0),
        ndes.MixtureDensityNetwork(n_parameters=10, n_data=ndata, n_components=1, n_hidden=[30,30], activations=[tf.tanh, tf.tanh], index=1),
        ndes.MixtureDensityNetwork(n_parameters=10, n_data=ndata, n_components=2, n_hidden=[30,30], activations=[tf.tanh, tf.tanh], index=2),
        ndes.MixtureDensityNetwork(n_parameters=10, n_data=ndata, n_components=3, n_hidden=[30,30], activations=[tf.tanh, tf.tanh], index=3),
        ndes.MixtureDensityNetwork(n_parameters=10, n_data=ndata, n_components=4, n_hidden=[30,30], activations=[tf.tanh, tf.tanh], index=4),
        ndes.MixtureDensityNetwork(n_parameters=10, n_data=ndata, n_components=5, n_hidden=[30,30], activations=[tf.tanh, tf.tanh], index=5)]


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



## Create DELFI object

In [ ]:
DelfiEnsemble = delfi.Delfi(compressed_data, prior, NDEs,
                       param_names = ['logKonTN', 'logKonTC', 'logKonRN', 'logKonRC', 'logKDTN', 'logKDTC', 'logKDRN', 'logKDRC', 'malpha', 'alpha0'],
                       results_dir = "results_batch_" + let + "/", save = False)

## Fisher pre-training to initialize the NDEs

## Sequential Neural Likelihood

## Sample the learned posterior

In [ ]:
#pst = DelfiEnsemble.emcee_sample()
pst = np.load('simulators/faas/pst_samples_' + let + '.npy')
time = np.genfromtxt('simulators/faas/data/time_points.csv', delimiter=',')
#sims = simulator(pst[1], seed, simulator_args, 0)
#data = faasSimulator.data()[batch_min:batch_max,:]

print(pst[49])
#fig,ax = plt.subplots(1,2) 

#for i in range(len(sims)):
#    ax[0].plot(time, data[i])
#    ax[1].plot(time, sims[i])


#plt.show()

In [ ]:
#pst = np.unique(pst, axis=0)
pst1 = pst[np.abs(pst[:,0]) < 13] #Discard any obviously wrong samples (write better func to do this)
print(pst1.shape)

In [ ]:
plt.scatter(np.log(pst[0,:]), np.log(pst[3,:]))
plt.show()

## Alright let's plot it!
Feed it a list of `(n_samples, n_parameters)` arrays for making a triangle plot; in this case let's just plot the posterior samples.

In [ ]:
DelfiEnsemble.triangle_plot(samples=[pst], savefig=True, filename='batch_' + let + '.pdf')

## Calibration

In [ ]:
# rank = np.zeros(10,9)

# for i in range(N):
#     ps = prior.draw()
#     sims = simulator(ps, 0, simulator_args, 1)
#     DelfiEnsemble = delfi.Delfi(sims, prior, NDEs, 
#                        param_limits = [lower, upper],
#                        param_names = ['logK_on_TN', 'logK_on_TC', 'logK_on_RN', 'logK_on_RC', 'logK_D_TN', 'logK_D_TC', 'logK_D_RN', 'logK_D_RC', 'm_alpha', 'alpha0'],
#                        results_dir = "simulators/faas/calib/", save = False)

    
    
#     n_initial = 100
#     n_batch = 10
#     n_populations = 1

#     DelfiEnsemble.sequential_training(simulator, compressor, n_initial, n_batch, n_populations, patience=2., save_intermediate_posteriors=False, simulator_args=simulator_args)
#     pst = DelfiEnsemble.emcee_sample()[:9]
#     for j in range(18):
#         for l in range(9):
#             rank[j,l] += np.sum(pst[l][j] < ps[j])
        
    
        

